<a href="https://colab.research.google.com/github/aimanhaziqmy/aimanhaziqmy/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fastapi nest-asyncio pyngrok uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 745 kB 56.2 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 80 kB 9.8 MB/s 
     |████████████████████████████████| 58 kB 6.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=e63ba7180238f417d0da8cb670b07ce06dea93720685e906f6d01def234c4cee
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [3]:
!ngrok authtoken 2EUPnEFbJWFPUHF201BuZy5tsbU_61FVe27674ENKqS12STYy

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
from enum import Enum

class Gender(str, Enum):
  male = "male"
  female = "female"


In [5]:
class Role(str, Enum):
  admin = "admin"
  user = "user"
  student = "student"

In [7]:
from pydantic import BaseModel
from uuid import UUID, uuid4
from typing import Optional,List

class User(BaseModel):
  id: Optional[UUID] = uuid4()
  first_name: str
  last_name: str
  middle_name: Optional[str]
  gender: Gender
  roles : List[Role]

In [8]:
class UserUpdateRequest(BaseModel) :
  first_name : Optional[str]
  last_name: Optional[str]
  middle_name : Optional[str]
  roles: Optional[str]

In [9]:
from fastapi.exceptions import HTTPException
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

db: List[User] = [User(id=UUID("b839b950-15be-497a-b4eb-eae05707ebb4"), first_name="Jamila", last_name="Ahmed",gender=Gender.female, roles=[Role.student]),
                  User(id=UUID("28b48120-36be-4662-a30b-1426d4958b4e"), first_name="Alex", last_name="Jones", gender=Gender.male, roles=[Role.admin, Role.user])]

@app.get('/')
async def root():
    return {'hello': 'world'}

@app.get("/api/v1/users")
async def fetch_users():
  return db;

@app.post("/api/v1/users")
async def register_user(user: User) :
  db.append(user)
  return {"id" : user.id}

@app.delete("/api/v1/users/{user_id}")
async def delete_user(user_id : UUID):
  for user in db:
    if user.id == user_id:
      db.remove(user)
      return 
  raise HTTPException(
      status_code=404,
      detail=f"user with id :{user_id} does not exists"
  )

@app.put("/api/v1/users/{user_id}")
async def update_user(user_update: UserUpdateRequest, user_id: UUID):
  for user in db:
    if user.id == user_id:
      if user_update.first_name is not None:
        user.first_name = user_update.first_name
      if user_update.last_name is not None:
        user.last_name = user_update.last_name
      if user_update.middle_name is not None:
        user.middle_name = user_update.middle_name
      if user_update.roles is not None:
        user.roles = user_update.roles
      return
  raise HTTPException(
      status_code = 404,
      detail=f"User with id : {user_id} does not exist"
  )


In [10]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

ERROR:pyngrok.process.ngrok:t=2022-09-08T15:01:40+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Your account is limited to 1 simultaneous ngrok agent session.\nActive ngrok agent sessions in region 'us':\n  - ts_2EUT73lKTHeyDRFQMBsZZXqW3Mh (34.125.197.71)\r\n\r\nERR_NGROK_108\r\n"


PyngrokNgrokError: ignored